In [1]:
import requests
import sqlite3
from sqlite3 import Error
import json

This notebook is for reading the sersver trip and matching the relevant user input with the phone UserCache's IMU data for each segment.

1. Connects to an instance of the e-mission server
2. Accepts a user UUID (e.g. email) and use it to fetch all processed trip data
3. For each trip segment, fetch all corresponding IMU data from the SQLite phone UserCache local database
4. Also for each trip segment, check and fetch user input ground truth from the UserCache.

For more details refer to https://github.com/e-mission/e-mission-docs/issues/627

In [2]:
# Replace these global variables with your actual setup.
SERVER_URL = "http://45.33.108.175:80"
LOOKUP_DATE = "2021-03-01"
USERNAME = "josh01"

post_msg = {
    "user": USERNAME
}

# To obtain the local database, use E-mission -> Profile -> Developer zone -> Check sensed data -> Email icon
# For more details, check https://github.com/e-mission/e-mission-docs/issues/613
PHONE_LOCAL_DATABASE_LOCATION = "/Users/bowenxu2/Downloads/userCacheDB"

In [3]:
# Helper method to create the phone local database connection.

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def select_last_row(conn, timestamp_key):
    """
    Find the last user input for the motion mode, given the timestamp_key.
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM userCache WHERE key='" + timestamp_key + "' LIMIT 5")

    rows = cur.fetchall()

    print("The number of user input results for the current segment motion mode:", len(rows))
#     for row in rows:    
#         print(row)
    if len(rows) > 0:
        last_row = rows[-1]
        print("last row:", last_row)
        print("latest user input:", json.loads(last_row[6])["label"])
        
def select_all_IMU_data_for_segment(conn, segment_start, segment_end, timestamp_offset_seconds=0):
    """
    Find all IMU data within a segment. The segement is defined by segment_start and segment_end.
    To help minimizing the possible trip segementation inaccuracy (i.e. the segment itself may be wrong), 
    I introduced a timestamp offset so that you can fetch IMU data at slightly larger/smaller periods than the actual segment. 
    So you can tweak the offset to get the ML prediction that's closest to the ground truth/user input.
    :param conn: the Connection object
    :param timestamp_offset_seconds: the offset. If positive, you read a longer period. If negative, you read a smaller period. 
    :return:
    """
    cur = conn.cursor()
    actual_segment_start = str(segment_start - timestamp_offset_seconds)
    actual_segment_end = str(segment_end + timestamp_offset_seconds)
    cur.execute("SELECT * FROM userCache WHERE key = 'background/movement_sensor' AND write_ts >= " 
                + actual_segment_start + 
                " AND write_ts <= " + actual_segment_end)

    rows = cur.fetchall()

    print("At offset", timestamp_offset_seconds, "seconds, number of IMU data rows:", len(rows))
#     for row in rows:    
#         print(row)
    if len(rows) > 0:
        for row in rows:
            print(row)

In [4]:
# Load the UserCache (SQLite database) and create the connection
conn = create_connection(PHONE_LOCAL_DATABASE_LOCATION)

In [5]:
def run_data_merging():
    # Get the E-mission server response.
    # The E-mission server uses a MongoDB (NoSQL), so there's no direct way of obtaining the dump containing all user data on all dates.
    # We need to use the POST API to obtain the daily data once at a time.
    response = requests.post(SERVER_URL + "/timeline/getTrips/" + LOOKUP_DATE, json=post_msg)
    
    today_trip_list = response.json()['timeline']
    print("today's number of trips:", len(today_trip_list))
    print("running data merging on date", LOOKUP_DATE)
    for idx, trip in enumerate(today_trip_list):
        print("Reviewing trip", idx + 1, "of", len(today_trip_list))
        print("trip start time:", trip['properties']['start_fmt_time'])
        print("trip end time:", trip['properties']['end_fmt_time'])
        print("Reading segment data...")
        segment_list = []
        for feature in trip['features']:
            if feature['type'] == "FeatureCollection":
                segment_list.append(feature)
                segment_start_timestamp = feature['features'][0]['properties']['start_ts']
                segment_end_timestamp = feature['features'][0]['properties']['end_ts']
                print("current segment start time:", feature['features'][0]['properties']['start_fmt_time'], segment_start_timestamp)
                print("current segment end time:", feature['features'][0]['properties']['end_fmt_time'], segment_end_timestamp)
                raw_motion_mode_text = feature['features'][0]['properties']['sensed_mode']
                print("current segment motion mode, detected by E-mission model:", raw_motion_mode_text.split(".")[1])
                # To find the matching user input in the local database, get the key by "StartTimeStamp_EndTimeStamp"
                select_last_row(conn, str(segment_start_timestamp) + "_" + str(segment_end_timestamp))
                select_all_IMU_data_for_segment(conn, segment_start_timestamp, segment_end_timestamp, 100)
                print("--")
        print("Number of segments in this trip:", len(segment_list))
        print("-" * 20)

### Note: The "user input" data below is only generated for testing purposes and do not reflect the ground truth.
### Note 2: The IMU data was not collected on 2021-03-01. See another example below for the IMU data merging.

In [6]:
run_data_merging()

today's number of trips: 6
running data merging on date 2021-03-01
Reviewing trip 1 of 6
trip start time: 2021-03-01T10:33:10-08:00
trip end time: 2021-03-01T10:59:50-08:00
Reading segment data...
current segment start time: 2021-03-01T10:33:10-08:00 1614623590.0
current segment end time: 2021-03-01T10:59:50-08:00 1614625190.0
current segment motion mode, detected by E-mission model: CAR
The number of user input results for the current segment motion mode: 0
At offset 100 seconds, number of IMU data rows: 0
--
Number of segments in this trip: 1
--------------------
Reviewing trip 2 of 6
trip start time: 2021-03-01T11:09:44.717288-08:00
trip end time: 2021-03-01T11:47:13.945000-08:00
Reading segment data...
current segment start time: 2021-03-01T11:09:44.717288-08:00 1614625784.717288
current segment end time: 2021-03-01T11:47:13.945000-08:00 1614628033.945
current segment motion mode, detected by E-mission model: BICYCLING
The number of user input results for the current segment motion

In [7]:
# This is an example where the E-mission data and the IMU data are merged on a segment level.
LOOKUP_DATE = "2021-03-12"
run_data_merging()

today's number of trips: 1
running data merging on date 2021-03-12
Reviewing trip 1 of 1
trip start time: 2021-03-12T10:25:35.824000-08:00
trip end time: 2021-03-12T10:26:37.728000-08:00
Reading segment data...
current segment start time: 2021-03-12T10:25:35.824000-08:00 1615573535.824
current segment end time: 2021-03-12T10:26:37.728000-08:00 1615573597.728
current segment motion mode, detected by E-mission model: BUS
The number of user input results for the current segment motion mode: 0
At offset 100 seconds, number of IMU data rows: 3
(1615573573.865, None, 'America/Los_Angeles', 'sensor-data', 'background/movement_sensor', None, '{"accuracy":3,"numberOfBumps":0,"threshold":50.0,"timeDuration":30}')
(1615573603.977, None, 'America/Los_Angeles', 'sensor-data', 'background/movement_sensor', None, '{"accuracy":3,"numberOfBumps":0,"threshold":50.0,"timeDuration":30}')
(1615573633.98, None, 'America/Los_Angeles', 'sensor-data', 'background/movement_sensor', None, '{"accuracy":3,"numberO